In [ ]:
import os
import typing
import datetime as dt

import psycopg2
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', lambda x: '%.9f' % x)

In [ ]:
conn_postgres = psycopg2.connect(os.environ['POSTGRES_DRIVER_STR'])

In [ ]:
def _min_max_spreads(df: pd.DataFrame) -> pd.DataFrame:
    min_ = df[df.symbol.apply(lambda x: x in common_symbols)] \
        .sort_values(by=['spread_percents'], ascending=True) \
        .drop_duplicates(subset=['exchange', 'symbol'])
    max_ = df[df.symbol.apply(lambda x: x in common_symbols)] \
        .sort_values(by=['spread_percents'], ascending=False) \
        .drop_duplicates(subset=['exchange', 'symbol'])
    return min_.merge(max_, on=['symbol', 'exchange']).assign(
        spread_max=lambda x: x.spread_percents_y - x.spread_percents_x,
    )

def _to_postgres(t: dt.datetime) -> str:
    return t.strftime('%Y-%m-%d %H:%M:%S')

def _spreads_df(get_df: typing.Callable[[str, str], pd.DataFrame]) -> pd.DataFrame:
    now = dt.datetime.strptime('07-29-2023', '%m-%d-%Y')
    spreads = []
    while now < dt.datetime.utcnow():
        start = _to_postgres(now)
        end = _to_postgres(now + dt.timedelta(hours=1, seconds=-1))
        now += dt.timedelta(hours=1)
        spreads_df = get_df(start=start, end=end)
        print('start: {}, end: {}, spreads_df.len: {}'.format(start, end, len(spreads_df)))
        if spreads_df.empty:
            continue
        min_max_spreads = spreads_df[lambda x: x.symbol.apply(lambda x: x in common_symbols)] \
            .pipe(_min_max_spreads)
        spreads.append(min_max_spreads)

    return pd.concat(spreads)

def _to_csv(df: pd.DataFrame, filename: str) -> None:
    spreads_v1_df \
        .groupby(by=['symbol', 'exchange']) \
        .agg(
            spread_percents_min=('spread_percents_x', min),
            spread_percents_max=('spread_percents_y', max),
            spread_max=('spread_max', max)
        ) \
        .sort_values(by=['spread_percents_max'], ascending=False) \
        .to_csv(filename)

In [ ]:
amount_df = pd.read_sql_query('''
    select symbol, exchange, count(*) as amount
    from monitoring_spread_v2
    group by symbol, exchange
''', conn_postgres)

binance_symbols = set(amount_df[lambda x: x.exchange == 'binance'].symbol)
gateio_symbols = set(amount_df[lambda x: x.exchange == 'gateio'].symbol)
common_symbols = binance_symbols.intersection(gateio_symbols)

### v1

In [ ]:
%%time

def _spreads_v1_df(start: str, end: str) -> pd.DataFrame:
    return pd.read_sql_query(f'''
        select
            symbol,
            exchange,
            cast(last_ask - last_bid as float) / last_ask * 100 as spread_percents,
            timestamp
        from monitoring_spread_v2
        where timestamp >= to_timestamp('{start}', 'YYYY-MM-DD HH24:MI:SS')
            and timestamp < to_timestamp('{end}', 'YYYY-MM-DD HH24:MI:SS')
    ''', conn_postgres)

spreads_v1_df = _spreads_df(get_df=_spreads_v1_df)

In [ ]:
filename = dt.datetime.utcnow().strftime('monitor-v1--%m-%d-%Y.csv')
spreads_v1_df.pipe(_to_csv, filename=filename)

### v2

In [ ]:
%%time

def _spreads_v2_df(start: str, end: str) -> pd.DataFrame:
    return pd.read_sql_query(f'''
        select
            symbol,
            exchange,
            cast(high_price - low_price as float) / high_price * 100 as spread_percents,
            timestamp
        from monitoring_spread_v2_tickers
        where timestamp >= to_timestamp('{start}', 'YYYY-MM-DD HH24:MI:SS')
            and timestamp < to_timestamp('{end}', 'YYYY-MM-DD HH24:MI:SS')
            and high_price != 0
            and low_price != 0
    ''', conn_postgres)

spreads_v2_df = _spreads_df(get_df=_spreads_v2_df)

In [ ]:
filename = dt.datetime.utcnow().strftime('monitor-v2--%m-%d-%Y.csv')
spreads_v2_df.pipe(_to_csv, filename=filename)